In [1]:
import npu
import numpy as np
from npu.build.appbuilder import AppBuilder
from npu.runtime import AppRunner
from npu.build.kernel import Kernel
import pylab as plt
import scipy

In [2]:
M     = 3          # Number of taps
P     = 16       # Number of 'branches', also fft length
W     = 1024       # Number of windows of length M*P in input time stream
n_int = 10          # Number of time integrations on output data
n_points=M*P
dt=np.complex64

In [3]:
def createKernel(kernel_name):
    kernel_path = f"./kernels/{kernel_name}.c"
    with open(kernel_path, 'r') as file:
        data = file.read()
    get_ipython().run_cell_magic('kernel', '', data)

def generateKernels(W, M, P, dt):
    # Frontend kernel
    createKernel("fft")
    fft_kernel.ports[0].array = np.ndarray([1, P], dtype=np.uint64)
    fft_kernel.ports[1].array = np.ndarray([1, P], dtype=np.uint64)
    fft_kernel.ports[2].array = np.ndarray([1, P], dtype=np.uint64)

    return [fft_kernel]


In [4]:
kernels = generateKernels(W, M, P, dt)

In [5]:
class SimpleApplication(AppBuilder):
    
    def __init__(self, kernels):
        self.frontend = kernels[0]
        super().__init__()

    def callgraph(self, x_in:np.ndarray, x_in2:np.ndarray, x_out:np.ndarray):
        rows = x_in.shape[0]
        size = x_in.shape[1]
        for row in range(rows):
            kernel_output = self.frontend(x_in[row], x_in2[row], size)
            x_out[row] = kernel_output

In [6]:
# Making the app
app_builder = SimpleApplication(kernels)

# Building the app
input_form = np.zeros(shape=(1, P), dtype=np.uint64)
input_form2 = np.zeros(shape=(1, P), dtype=np.uint64)
output_form = np.zeros(shape=(1, P), dtype=np.uint64)

app_builder.build(input_form, input_form2, output_form)

app = AppRunner('SimpleApplication.xclbin')

Building the fft_kernel kernel...
ERROR! WSL failed 

RTNETLINK answers: File exists
In file included from fft_kernel.cc:8:
In file included from /opt/tools/Vitis/2023.1/aietools/include/aie_api/aie.hpp:85:
In file included from /opt/tools/Vitis/2023.1/aietools/include/aie_api/detail/abs.hpp:103:
In file included from /opt/tools/Vitis/2023.1/aietools/include/aie_api/detail/aie2/abs.hpp:54:
In file included from /opt/tools/Vitis/2023.1/aietools/include/aie_api/detail/aie2/../broadcast.hpp:196:
/opt/tools/Vitis/2023.1/aietools/include/aie_api/detail/aie2/broadcast.hpp:213:77: warning: 'clr32' is deprecated: Function 'clr' is deprecated. Please use the 'broadcast_zero_to' variant instead. [-Wdeprecated-declarations]
            if constexpr (Class == AccumClass::Int)  return []() { return ::clr32();  };
                                                                            ^
/opt/tools/Vitis/2023.1/aietools/data/aie_ml/lib/isg/me_chess_llvm.h:112034:7: note: 'clr32' has been explicit

CalledProcessError: Command '['bash', '/tmp/pnxjyll4pej/build_template/kernel_build.sh', 'fft_kernel']' returned non-zero exit status 255.

In [ ]:
def generate_win_coeffs(M, P, window_fn="hamming"):
    win_coeffs = scipy.signal.get_window(window_fn, P)
    sinc       = scipy.signal.firwin(P, cutoff=1.0/P, window="rectangular")
    win_coeffs *= sinc
    return win_coeffs

In [ ]:
# Generating input data
noise = np.random.normal(loc=0.5, scale=0.1, size=P).astype(np.float32)
samples = np.arange(P)
freq    = 1
amp     = 1
cw_signal = (amp * np.sin(samples * freq))
data = noise + cw_signal
plt.plot(data)
plt.show()
plt.clf()
data = data.astype(dt)

In [ ]:
# Generate window coefficients
win_coeffs = generate_win_coeffs(M, P, "hamming")
pg = np.sum(np.abs(win_coeffs)**2)
win_coeffs /= pg**.5 # Normalize for processing gain
win_coeffs = win_coeffs.astype(np.float32)
plt.plot(win_coeffs)
plt.show()
plt.clf()

In [ ]:
# Generating Twiddle groups
tw_groups = []
for i in 2 ** np.arange(np.log2(P) + 1):
    tw = np.tile(np.exp(-2j*np.pi*np.arange(i)/ i), int(P/i)).astype(dt)
    tw_groups.append(tw)

In [ ]:
d = tw_groups[0]
# extract real part 
x = [ele.real for ele in d] 
# extract imaginary part 
y = [ele.imag for ele in d] 
  
# plot the complex numbers 
plt.scatter(x, y) 
plt.ylabel('Imaginary') 
plt.xlabel('Real') 
plt.show() 
plt.clf()

In [ ]:
# Making app runner and running app
input_data = app.allocate(shape=(1, P), dtype=dt)
input_data2 = app.allocate(shape=(1, P), dtype=dt)
output_data = app.allocate(shape=(1, P), dtype=dt)

input_data[:] = data
input_data.sync_to_npu()
input_data2[:] = tw_groups[0]
input_data2.sync_to_npu()

app.call(input_data, input_data2, output_data)

output_data.sync_from_npu()

In [ ]:
plt.plot(output_data[0])
plt.show()
plt.clf()

In [ ]:
del app